<!-- ---
title: "User Guide"
sidebar: python
format:
  html:
    toc: true

---

# py-distance-transforms User Guide

This guide covers both basic and advanced usage of the py-distance-transforms library.

## Getting Started

### Installation

```bash
pip install py_distance_transforms
```

For GPU support, make sure you have PyTorch with CUDA installed:

```bash
pip install torch --extra-index-url https://download.pytorch.org/whl/cu118
```

### Basic Usage

The primary function in py_distance_transforms is `transform`. This function takes a binary array (with 0s and 1s) and calculates the squared Euclidean distance from each 0 pixel to the nearest 1 pixel.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from py_distance_transforms import transform

# Create a random binary array
arr = np.random.choice([0, 1], size=(10, 10)).astype(np.float32)

# Apply distance transform
result = transform(arr)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.imshow(arr, cmap='gray')
ax1.set_title('Original')
ax2.imshow(result, cmap='gray')
ax2.set_title('Distance Transform')
plt.tight_layout()
plt.show()

### Real-World Example

Let's load an image and apply a distance transform:


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from py_distance_transforms import transform
from skimage import io, color, filters

# Download and load image
import urllib.request
url = "http://docs.opencv.org/3.1.0/water_coins.jpg"
urllib.request.urlretrieve(url, "coins.jpg")
img = io.imread("coins.jpg")

# Convert to grayscale and threshold
img_gray = color.rgb2gray(img)
img_binary = img_gray > filters.threshold_otsu(img_gray)

# Apply distance transform
img_dist = transform(img_binary.astype(np.float32))

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(img)
axes[0].set_title('Original Image')
axes[1].imshow(img_binary, cmap='gray')
axes[1].set_title('Binary Image')
axes[2].imshow(img_dist, cmap='viridis')
axes[2].set_title('Distance Transform')
for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

### Understanding Euclidean Distance

By default, `transform` returns the squared Euclidean distance. For the true Euclidean distance, take the square root:


In [ ]:
import numpy as np
from py_distance_transforms import transform

# Create a simple array
arr = np.array([
    [0, 1, 1, 0, 1],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 0, 0]
], dtype=np.float32)

# Apply squared Euclidean distance transform
sq_dist = transform(arr)

# Calculate true Euclidean distance
euc_dist = np.sqrt(sq_dist)

print("Original array:")
print(arr)
print("\nSquared Euclidean distance:")
print(sq_dist)
print("\nEuclidean distance:")
print(euc_dist)

### Comparison with SciPy

Let's compare with SciPy's implementation:


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from py_distance_transforms import transform
from scipy.ndimage import distance_transform_edt
import time

# Create a larger random binary array
size = 100
arr = np.random.choice([0, 1], size=(size, size)).astype(np.float32)

# Time SciPy implementation
start = time.time()
scipy_result = distance_transform_edt(arr)**2  # Square to match our output
scipy_time = time.time() - start

# Time our implementation
start = time.time()
our_result = transform(arr)
our_time = time.time() - start

# Calculate difference
diff = np.abs(scipy_result - our_result)

print(f"SciPy time: {scipy_time:.6f} seconds")
print(f"Our time: {our_time:.6f} seconds")
print(f"Speedup: {scipy_time/our_time:.2f}x")
print(f"Max difference: {np.max(diff):.8f}")

# Visualize results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(scipy_result, cmap='viridis')
axes[0].set_title('SciPy Result')
axes[1].imshow(our_result, cmap='viridis')
axes[1].set_title('Our Result')
axes[2].imshow(diff, cmap='hot')
axes[2].set_title('Difference')
plt.tight_layout()
plt.show()

## Advanced Features

### GPU Acceleration

py_distance_transforms supports GPU acceleration for NVIDIA GPUs using PyTorch tensors.


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
from py_distance_transforms import transform_cuda

# Check if CUDA is available
if torch.cuda.is_available():
    # Create a random binary tensor on GPU
    size = 1000
    x_gpu = torch.rand((size, size), device='cuda')
    x_gpu = (x_gpu > 0.5).float()
    
    # Measure time for GPU computation
    start = time.time()
    result_gpu = transform_cuda(x_gpu)
    gpu_time = time.time() - start
    
    # Transfer to CPU for comparison
    x_cpu = x_gpu.cpu().numpy()
    
    # Measure time for CPU computation
    start = time.time()
    result_cpu = transform(x_cpu)
    cpu_time = time.time() - start
    
    print(f"CPU time: {cpu_time:.6f} seconds")
    print(f"GPU time: {gpu_time:.6f} seconds")
    print(f"Speedup: {cpu_time/gpu_time:.2f}x")
    
    # Compare results
    result_from_gpu = result_gpu.cpu().numpy()
    diff = np.abs(result_cpu - result_from_gpu)
    print(f"Max difference: {np.max(diff):.8f}")
else:
    print("CUDA is not available. GPU examples cannot be run.")

### Working with 3D Data

py_distance_transforms supports 3D arrays as well:


import numpy as np
import matplotlib.pyplot as plt
from py_distance_transforms import transform
from mpl_toolkits.mplot3d import Axes3D

# Create a 3D binary array
size = 20
arr_3d = np.zeros((size, size, size), dtype=np.float32)
# Add a few 1s
arr_3d[5, 5, 5] = 1
arr_3d[15, 15, 15] = 1
arr_3d[5, 15, 10] = 1

# Apply distance transform
result_3d = transform(arr_3d)

# Visualize a slice
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
slice_idx = size // 2
ax1.imshow(arr_3 -->